# Reproducing Calvano et al. (2020)
## Baseline
### Author: Andréa Epivent

In [1]:
# Import relevant packages
import os
import numpy as np
import pandas as pd
import random
from IPython.display import clear_output
from copy import deepcopy
import seaborn as sns
sns.set()

In [2]:
# Import our custom functions
os.chdir("/Users/admin/Desktop/ENSAE/3A/Mémoire/Codes/Functions")
import find_state
import profitquantity

### Initialization

* Parameters

In [3]:
n = 2
m = 15
ci = 1
ai = 2
a0 = 0
mu = 1/4
delta = 0.95
m = 15
epsilon = 0.1
k = 1 # memory width 
p_N = 1.47 # has to be solved numerically
p_M = 1.92 # has to be solved numerically

state_space = m**(n*k)
action_space = m

* Action space

In [4]:
# m equally spaced points in an interval that includes Nash and monopoly prices
A = np.linspace(p_N-epsilon*(p_M-p_N),p_M+epsilon*(p_M-p_N),m)
print(A.shape)

(15,)


* State space

In [5]:
# Combination of prices from last period
S = np.zeros([state_space, 2])
l = 0
for i in A:
    for j in A:
        S[l,0] = i
        S[l,1] = j
        l += 1

In [6]:
np.save('/Users/admin/Desktop/ENSAE/3A/Mémoire/Codes/Output/Baseline/actions', A)
np.save('/Users/admin/Desktop/ENSAE/3A/Mémoire/Codes/Output/Baseline/states', S)

* Q-matrix

In [7]:
# We start with zeros
q_table = np.zeros([state_space, action_space])
print(q_table.shape)

(225, 15)


In [8]:
init_Q()

b = 0 # loop over column
sum_profit = 0
for l in range(state_space):
    for i in A:
        for j in A:
            profit = profitquantity.profit_compute(i,j,ci,ai,mu,a0)
            sum_profit += profit
        denom = (1-delta)*(action_space**(n-1))
        q_table[l,b] = sum_profit/denom
        sum_profit = 0
        b += 1
    b = 0

### Training

In [17]:
%%time
"""Training the agent"""

# Hyperparameters
alpha = 0.15 # learning rate - baseline scenario
beta = 4*10**-6 # experimentation parameter - baseline scenario

# Stop criterion
criterion = 10**4

# Stop in any case
criterion_final = 15*10**5 

# Number of episodes
n_episodes = 100

# Store info in array
q_info = np.zeros((criterion_final,4*n_episodes))

# Initial Q_tables
q_tables1 = np.zeros([state_space, action_space])
q_tables2 = np.zeros([state_space, action_space])

for j in range(n_episodes):

    # Initialize Q-tables
    q_table_a1 = deepcopy(q_table)
    q_table_a2 = deepcopy(q_table)

    # Stop criterion
    count_a1 = 0
    count_a2 = 0

    # The initial state is picked randomly
    state = random.randint(0, state_space-1)

    # Store initial state in dataframe
    q_info[0,j*4] = S[state][0]
    q_info[0,(j*4)+1] = S[state][1]

    # Initialize matrix for keeping track of argmax_p q
    stab1 = np.full([state_space],-1)
    stab2 = np.full([state_space],-1)

    # Initialize convergence
    convergence = False

    # Start iteration
    i = 1    

    # While we didn't reach convergence
    while convergence == False:

        # Time-declining exploration rate
        epsilon = np.exp(-beta*i) # greedy parameter 

        # trade-off for agent 1
        if random.uniform(0, 1) < epsilon:
            action_a1 = random.randint(0, action_space-1) # Explore action space
        else:
            action_a1 = np.argmax(q_table_a1[state]) # Exploit learned values

        # trade-off for agent 2
        if random.uniform(0, 1) < epsilon:
            action_a2 = random.randint(0, action_space-1) # Explore action space
        else:
            action_a2 = np.argmax(q_table_a2[state]) # Exploit learned values

        # Retrieve new prices and next state
        p1, p2 = A[action_a1], A[action_a2]
        next_state = find_state.find_rowindex(S,p1,p2) # We find the row index associated with these two new prices

        # Rewards
        reward_a1 = profitquantity.profit_compute(p1,p2,ci,ai,mu,a0)
        reward_a2 = profitquantity.profit_compute(p2,p1,ci,ai,mu,a0)

        # Store in dataframe - We begin at i = 1
        q_info[i,j*4] = p1
        q_info[i,(j*4)+1] = p2
        q_info[i,(j*4)+2] = reward_a1
        q_info[i,(j*4)+3] = reward_a2

        # Convergence
        a1_argmax = np.argmax(q_table_a1[state]) 
        a2_argmax = np.argmax(q_table_a2[state]) 

        if a1_argmax == stab1[state]:
            count_a1 += 1
        else:
            count_a1 = 0
            stab1[state] = a1_argmax # reinitialization

        if a2_argmax == stab2[state]:
            count_a2 += 1
        else:
            count_a2 = 0
            stab2[state] = a2_argmax

        if (count_a1 >= criterion) & (count_a2 >= criterion):
            convergence = True

        # Updating Q-table
        # Agent 1
        old_value_a1 = q_table_a1[state, action_a1]
        next_max_a1 = np.max(q_table_a1[next_state])

        new_value_a1 = (1 - alpha) * old_value_a1 + alpha * (reward_a1 + delta * next_max_a1)
        q_table_a1[state, action_a1] = new_value_a1

        # Agent 2
        old_value_a2 = q_table_a2[state, action_a2]
        next_max_a2 = np.max(q_table_a2[next_state])

        new_value_a2 = (1 - alpha) * old_value_a2 + alpha * (reward_a2 + delta * next_max_a2)
        q_table_a2[state, action_a2] = new_value_a2

        # Go to next step
        state = next_state

        # Display number of episodes and status of convergence
        if i % 100 == 0:
            clear_output(wait=True)
            print(f"Iteration: {i}")
            print(f"Episode: {j}")

        if (i < criterion_final) & (convergence == True):
            print("Process has converged")

        # If we didn't convergence after a certain threshold, we end the loop anyway
        if (i == criterion_final-1) & (convergence == False):
            print("Process has not converged") 
            convergence = True

        i += 1
        
    # Save every Q-table
    q_tables1 = np.concatenate((q_tables1,q_table_a1))
    q_tables2 = np.concatenate((q_tables2,q_table_a2))

    print(f"Training finished, episode: {j}")

Iteration: 1314600
Episode: 99
Process has converged
Training finished, episode: 99
CPU times: user 7h 4min 34s, sys: 55min 35s, total: 8h 9s
Wall time: 8h 13min 59s


In [18]:
# Store results
# Information on prices and profits for both agents
np.save('/Users/admin/Desktop/ENSAE/3A/Mémoire/Codes/Output/Baseline/q_info', q_info)

# Last Q matrix of both agents
np.save('/Users/admin/Desktop/ENSAE/3A/Mémoire/Codes/Output/Baseline/q_table_a1', q_tables1)
np.save('/Users/admin/Desktop/ENSAE/3A/Mémoire/Codes/Output/Baseline/q_table_a2', q_tables2)

In [2]:
q_info = np.load('/Users/admin/Desktop/ENSAE/3A/Mémoire/Codes/Output/Baseline/q_info.npy')

### Formatting

In [5]:
# Deal with column names
col = ["p1_ep","p2_ep","profit1_ep","profit2_ep"]
newcol = []
for i in range(n_episodes):
    for j in col:
        newcol.append(j+str(i+1))
        
# Create dataframe
q_df = pd.DataFrame(q_info, columns=newcol)

In [8]:
# Find convergence threshold
for i in range(n_episodes):
    name = "convergence_ep"+str(i+1)
    if q_df['p1_ep'+str(i+1)].iloc[criterion_final-1] != 0:
        q_df[name] = criterion_final
    else:
        q_df[name] = q_df.index[q_df['p1_ep'+str(i+1)]==0][0]

convergence = np.array(q_df.iloc[0,400:500])
convergence.mean()

1141930.59

In [20]:
# Stock last prices
price1 = []
price2 = []
for i in range(n_episodes):
    price1.append(q_df["p1_ep"+str(i+1)].iloc[q_df["convergence_ep"+str(i+1)].iloc[0]-1])
    price2.append(q_df["p2_ep"+str(i+1)].iloc[q_df["convergence_ep"+str(i+1)].iloc[0]-1])
    
print(np.mean(price1))
print(np.mean(price2))

1.7790857142857142
1.773685714285714


In [22]:
# Compute extra-profit gains - more efficiently
profit_N = profitquantity.profit_compute(p_N,p_N,ci,ai,mu,a0)
profit_M = profitquantity.profit_compute(p_M,p_M,ci,ai,mu,a0)

extra_profit = []
for i in range(len(price1)):
    extra_profit.append((((profitquantity.profit_compute(price1[i],price2[i],ci,ai,mu,a0) + profitquantity.profit_compute(price2[i],price1[i],ci,ai,mu,a0))/2)-profit_N)/(profit_M-profit_N))

In [24]:
# Save as numpy format for later use
np.save('/Users/admin/Desktop/ENSAE/3A/Mémoire/Codes/Output/Baseline/convergence', convergence)
np.save('/Users/admin/Desktop/ENSAE/3A/Mémoire/Codes/Output/Baseline/price1', np.array(price1))
np.save('/Users/admin/Desktop/ENSAE/3A/Mémoire/Codes/Output/Baseline/price2', np.array(price2))
np.save('/Users/admin/Desktop/ENSAE/3A/Mémoire/Codes/Output/Baseline/extra_profit', np.array(extra_profit))